# Import libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns  # for charts
import matplotlib as mpl  # plotting
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import os
from collections import defaultdict
import re
import random
from pgmpy.models import BayesianModel


P:\conda\envs\cv2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
concatenated_text = ""
directory_path = r"C:\Users\ehabe\Desktop\poets"


# Loop through each file in the specified directory


In [3]:
for filename in os.listdir(directory_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(directory_path, filename)

        with open(file_path, 'r', encoding='utf-8') as file:
            poetry_text = file.read()

        concatenated_text += poetry_text + "\n"  # Add a newline between each poet's text



# Using regular expression to find all and lower all words the text


In [4]:
all_poetry_text = re.findall(r'\w+', concatenated_text.lower())
print(all_poetry_text[:100])


['looking', 'for', 'some', 'education', 'made', 'my', 'way', 'into', 'the', 'night', 'all', 'that', 'bullshit', 'conversation', 'baby', 'can', 't', 'you', 'read', 'the', 'signs', 'i', 'won', 't', 'bore', 'you', 'with', 'the', 'details', 'baby', 'i', 'don', 't', 'even', 'wanna', 'waste', 'your', 'time', 'let', 's', 'just', 'say', 'that', 'maybe', 'you', 'could', 'help', 'me', 'ease', 'my', 'mind', 'i', 'ain', 't', 'mr', 'right', 'but', 'if', 'you', 're', 'looking', 'for', 'fast', 'love', 'if', 'that', 's', 'love', 'in', 'your', 'eyes', 'it', 's', 'more', 'than', 'enough', 'had', 'some', 'bad', 'love', 'so', 'fast', 'love', 'is', 'all', 'that', 'i', 've', 'got', 'on', 'my', 'mind', 'ooh', 'ooh', 'ooh', 'ooh', 'looking', 'for', 'some', 'affirmation']


# Markov Chain

## Create a dictionary to store word transitions and their probabilities


In [5]:
markov_chain = defaultdict(lambda: defaultdict(int))

## Count Words for the markov_chain dictionary


In [6]:
    for i in range(len(all_poetry_text) - 1):
        current_word = all_poetry_text[i]
        next_word = all_poetry_text[i + 1]
        markov_chain[current_word][next_word] += 1


## Convert counts to probabilities


In [7]:
    for current_word, next_words in markov_chain.items():
        total_occurrences = sum(next_words.values())
        for next_word in next_words:
            next_words[next_word] /= total_occurrences


## User input for the first word and poem length


In [8]:
user_input = input("Enter the first word for the poem : ")
start_word = user_input
user_input = input("Enter the length for the poem : ")
poem_length = user_input

Enter the first word for the poem :  ali
Enter the length for the poem :  50


## Predict poems

In [9]:
poem_length=50
current_word= start_word
poem = [current_word]
    
for _ in range(poem_length - 1):
    current_word = poem[-1]
        
    if current_word in markov_chain:
        next_words = markov_chain[current_word]
            
        next_word = random.choices(list(next_words.keys()), weights=list(next_words.values()))[0]
            
            # Append the next word to the poem
        poem.append(next_word)
    else:
            # If the current word has no next words, end the poem
        break


# Example usage
print(poem)


['ali', 'you', 'doo', 'just', 'change', 'the', 'church', 'steps', 'till', 'it', 'strong', 'i', 'ain', 't', 'you', 'would', 'we', 'can', 'heal', 'the', 'trees', 'by', 'myself', 'better', 'come', 'after', 'setting', 'sun', 'he', 'll', 'never', 'taught', 'me', 'down', 'upon', 'the', 'fiends', 'i', 'let', 's', 'joy', 'the', 'air', 'it', 'was', 'never', 'living', 'as', 'if', 'this']


## Follow poet style

In [10]:
def poet_style(word_list):
    for i in range(0, (poem_length - 1), 15):
    
        lines = [
        " ".join(word_list[i:i+3]),
        " ".join(word_list[i+3:i+6]),
        " ".join(word_list[i+6:i+9]),
        " ".join(word_list[i+9:i+12]),
        " ".join(word_list[i+12:i+15])
        ]

        # Repeat the lines for emphasis
        repeated_lines = lines * 2

        # Print the poem
        for line in repeated_lines:
            print(line)



In [11]:
poet_style(poem)

ali you doo
just change the
church steps till
it strong i
ain t you
ali you doo
just change the
church steps till
it strong i
ain t you
would we can
heal the trees
by myself better
come after setting
sun he ll
would we can
heal the trees
by myself better
come after setting
sun he ll
never taught me
down upon the
fiends i let
s joy the
air it was
never taught me
down upon the
fiends i let
s joy the
air it was
never living as
if this



never living as
if this





# Bayesian network

In [12]:
def build_counts(all_poetry_text, number_of_words=2):
    transitions = {}

    for i in range(len(all_poetry_text) - number_of_words):
        current_state = tuple(all_poetry_text[i:i + number_of_words])
        next_state = all_poetry_text[i + number_of_words]

        transitions[current_state] = transitions.get(current_state, {})
        transitions[current_state][next_state] = transitions[current_state].get(next_state, 0) + 1

    return transitions


In [13]:
def make_bayesian_network(transitions):
    model = BayesianModel()
    
    for current_state, next_states in transitions.items():
        for next_state, count in next_states.items():
            model.add_edge(current_state, next_state, weight=count)
    
    return model

In [14]:
def generate_poem(bayesian_network, limit, start):
    n = 0
    curr_state = start
    poem = f"{curr_state[0]} {curr_state[1]}"

    while n < limit:
        next_state = random.choices(
            list(bayesian_network.neighbors(curr_state)),
            weights=[bayesian_network.get_edge_data(curr_state, neighbor)['weight'] for neighbor in bayesian_network.neighbors(curr_state)]
        )[0]
        poem += f" {next_state}"
        curr_state = (curr_state[1], next_state)
        n += 1

    return poem

In [15]:
countss = build_counts(all_poetry_text)
bayesian_network = make_bayesian_network(countss)

In [18]:
# User input
start_word_input = input("Enter the first two words for the poem: ")
start_words = start_word_input.split()[:2]  

poem_length_input = input("Enter the length for the poem: ")
poem_length = int(poem_length_input)


Enter the first two words for the poem:  i love 
Enter the length for the poem:  50


In [19]:
generated_poem = generate_poem(bayesian_network, poem_length, tuple(start_words))


In [20]:
new_poem = re.findall(r'\w+', generated_poem.lower())

In [21]:
poet_style(new_poem)

i love you
no one gives
me everything i
wanna say the
weak or the
i love you
no one gives
me everything i
wanna say the
weak or the
marine that went
to a plane
so we could
belong together artpop
we could we
marine that went
to a plane
so we could
belong together artpop
we could we
could save a
cent more for
piguets boardin a
jet big booty
heifer gettin by
could save a
cent more for
piguets boardin a
jet big booty
heifer gettin by
it s more
up there on
my


it s more
up there on
my


